In [1]:
# RUN THIS CELL: it loads some style files
from IPython.core.display import HTML, display, Math
with open( './style/custom.css', 'r' ) as f: html_style = f.read()
HTML( html_style )

# Test binomiale

Un'urna contiene monete equilibrate e monete difettose. Le monete equilibrate hanno probabilità di successo $p=1/2$ le monete difettose hanno probabilità di successo ignota $p>1/2$. Non conosciamo la frazione di monete difettose. Nel gergo dei <mark>test di ipotesi</mark> riassumeremo questa situazione scrivendo:

$H_0:\kern3.5ex p=1/2\quad$ <mark>ipotesi nulla</mark>

$H_A:\kern3ex p>1/2\quad$ <mark>ipotesi alternativa</mark>
 
In questo modo dichiariamo che $p=1/2$ è ciò che consideraimo *normale* o *sano* e che  $p>1/2$ è il tipo di deviazione dalla normalità o *patologia* che vogliamo rilevare.

Estraiamo una moneta dall'urna e, per decidere tra equilibrata o difettosa, facciamo il seguente test: la lanciamo $20$ volte e se il numero dei successi è $\ge k$ la dichiariamo difettosa. Stiamo descivendo una famiglia di test, uno per ogni scelta del parametro $k$. Questo parametro lo decide chi progetta il test cercando di minimizzare gli errori. Vogliamo per prima cosa come varia la probabilità di falsi positivi/negativi al variare di $k$. 

Il test è una variabile aleatoria $X$ a valori in $\{0,\dots,20\}$. Lo spazio campionario $\Omega$ è diviso in due parti: $H_A$ e $H_0$.  L'insieme $H_A$ contiene quegli $\omega$ che corrispondono a $20$ lanci fatti con una moneta difettosa mentre $H_0$ contiene quegli $\omega$ che corrispondono a lanci con una moneta equilibrata. 

Condizionando a $H_0$ otteniamo $X\sim {\rm B}(20,\ 1/2)$. Condizionando a $H_A$ otteniamo $X\sim {\rm B}(20,\ p)$ con $p>1/2$ ignota.

In [2]:
from scipy.stats import binom                             # lib. for statistical functions
from ipywidgets import interact, FloatSlider, IntSlider   # lib. for interactive graphic
from bokeh.io import push_notebook, show, output_notebook # lib. for graphic output
from bokeh.plotting import figure
output_notebook()
options = dict(plot_height=250,plot_width=700,
               tools="pan,wheel_zoom,reset,save,crosshair,box_select")
def printred(s):
    display(HTML('<pre style="font: bold 12pt Courier, serif;color:#a02">' + s + '</pre>') )
def printgreen(s):
    display(HTML('<pre style="font: bold 12pt Courier, serif;color:#080">' + s + '</pre>') )

Loading BokehJS ...

In [3]:
n = 20; p = 1/2; k_max = 15
x = range(0,n+1)
# create an empty figure
plot = figure(title="PMF of X ~ B(20, p).   The interval [k, 20] is hightlighted.", 
              x_axis_label = "# successes", y_axis_label = "probability", 
              x_range=(0,n+1), y_range=(0,0.25), 
              **options )
plot.title.text_font="times"
plot.title.text_font_size="14pt"

# initialize and show barplot
r1 = plot.vbar(x, top=[0 for i in x], bottom=0, width=.9, color='green', alpha=0.5)
r2 = plot.vbar(x, top=[0 for i in x], bottom=0, width=.9, color='red', alpha=0.5)
show(plot, notebook_handle=True)

def s1(p):
    if p == 0.5: return '= α = significatività = Pr( test positivo | moneta equilibrata )' 
    else: return ''
def s2(p):
    if p > 0.5: return '= β = Pr( test negativo | moneta sbilanciata )' 
    else: return ''
        
def update(k, p):
    data = {'x':x, 'top':  binom.pmf(x, n, p) }
    x_rif = range(k,n+1)
    data2 = {'x':x_rif, 'top':  binom.pmf(x_rif, n, p) }
    r1.data_source.data = data
    r2.data_source.data = data2
    printred('T = [ {}, 20 ]   zona di rifiuto'.format(k) )
    printred('Pr( T | p = {:0.1f} ) = {:0.3f} '.format(p,binom.pmf(k-1, n, p)) + s1(p))  
    printgreen('Pr(¬T | p = {:0.1f} ) = {:0.3f} '.format(p,1-binom.pmf(k-1, n, p)) + s2(p) )
    push_notebook()
    
interact(update,
         k = IntSlider  (description="k", min=10, max=k_max, step=1, value=n), 
         p = FloatSlider(description="p", min=0.5, max=0.8, step=0.1, value=0.5));

interactive(children=(IntSlider(value=15, description='k', max=15, min=10), FloatSlider(value=0.5, description…

# Osservazione

$\alpha$ si chiama <mark>significatività</mark>

$\alpha$ misura la probabilità si un errore del <mark>I tipo</mark> (falso positivo)

$\beta$ non ha nome, ma $1-\beta$ si chiama <mark>potenza</mark>, nei test diagnostici si chiamava *sensitività*. 

$\beta$ misura la probabilità si un errore del <mark>II tipo</mark> (falso negativo). Si noti però che $\beta$ è ben definito: varia  al variare di $p$. Di solito si prende il massimo possibile.

